In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize, claptonize_opt, claptonize_opt_scipy, claptonize_opt_ng, claptonize_opt_structured
from clapton.ansatzes import circular_ansatz, full_ansatz
from clapton.utils import Results
import numpy as np
import pickle
import time
import copy


from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem

from optuna.samplers import TPESampler, RandomSampler, GridSampler, CmaEsSampler, QMCSampler, GPSampler, NSGAIIISampler, NSGAIISampler


import nevergrad as ng
# from nevergrad.families import NonObjectOptimizer
# from nevergrad.optimizers import NGOpt
# from nevergrad.optimization.optimizerlib import EDA, ConfPSO, ParametrizedOnePlusOne, cGA, NoisySplit, MultiDiscrete, AXP, BFGSCMAPlus, CMA, CSEC11, Carola3, ChoiceBase, ForceMultiCobyla, \
#     LogMultiBFGSPlus, MEDA, MetaCMA, MultiBFGSPlus, MultiCobylaPlus, MultiSQPPlus, NGO, PCEDA, SPSA, SQPCMAPlus, Shiwa, SplitOptimizer, Wiz


In [2]:
import gurobipy as gp
from gurobipy import GRB

def optimize_stabilizer(n, pauli_XZ, coeffs):
    """
    n: number of qubits
    pauli_XZ: list of tuples (x_vec,z_vec) in {0,1}^n
    coeffs: list of real c_t
    """
    m = len(coeffs)
    model = gp.Model("best_stabilizer_state")
    M = n

    # 1) Tableau vars
    x = model.addVars(n, n, vtype=GRB.BINARY, name="x")
    z = model.addVars(n, n, vtype=GRB.BINARY, name="z")
    r = model.addVars(n, vtype=GRB.BINARY,    name="r")

    # 2) Commutation slack
    u = model.addVars(n, n, vtype=GRB.INTEGER, name="u")

    # 3) Membership & expectation vars
    vsel   = model.addVars(m, n, vtype=GRB.BINARY,  name="vsel")
    delta  = model.addVars(m, n, vtype=GRB.INTEGER, name="delta")
    eps    = model.addVars(m, n, vtype=GRB.INTEGER, name="eps")
    y      = model.addVars(m,    vtype=GRB.BINARY,  name="y")
    sigma  = model.addVars(m,    vtype=GRB.INTEGER, name="sigma")
    zeta   = model.addVars(m,    vtype=GRB.BINARY,  name="zeta")
    p_var  = model.addVars(m,    vtype=GRB.BINARY,  name="p")
    q_var  = model.addVars(m,    vtype=GRB.BINARY,  name="q")

    # --- Commutation constraints ---
    for i in range(n):
        for k in range(i+1, n):
            model.addConstr(
               gp.quicksum(x[i,j]*z[k,j] + z[i,j]*x[k,j] for j in range(n))
               - 2*u[i,k] == 0
            )

    # --- Membership and sign constraints ---
    for t, ((x_t, z_t), c_t) in enumerate(zip(pauli_XZ, coeffs)):
        # parity‐match on X and Z (big M trick)
        for j in range(n):
            expr_x = gp.quicksum(vsel[t,i]*x[i,j] for i in range(n)) - 2*delta[t,j] - x_t[j]
            expr_z = gp.quicksum(vsel[t,i]*z[i,j] for i in range(n)) - 2*eps[t,j]   - z_t[j]
            model.addConstr( expr_x <=  M*(1-y[t]) )
            model.addConstr(-expr_x <=  M*(1-y[t]) )
            model.addConstr( expr_z <=  M*(1-y[t]) )
            model.addConstr(-expr_z <=  M*(1-y[t]) )

        # phase parity
        model.addConstr(
            gp.quicksum(vsel[t,i]*r[i] for i in range(n))
            - 2*sigma[t] - zeta[t] == 0
        )

        # link p,q to y and zeta
        model.addConstr(p_var[t] + q_var[t] == y[t])
        model.addConstr(p_var[t] <= 1 - zeta[t])
        model.addConstr(q_var[t] <=     zeta[t])

    # --- Objective ---
    obj = gp.quicksum(coeffs[t]*(p_var[t] - q_var[t]) for t in range(m))
    model.setObjective(obj, GRB.MINIMIZE)

    # --- Optimize ---
    model.Params.OutputFlag = 1
    model.optimize()

    # retrieve tableau
    X_sol = [[int(x[i,j].X) for j in range(n)] for i in range(n)]
    Z_sol = [[int(z[i,j].X) for j in range(n)] for i in range(n)]
    r_sol = [int(r[i].X) for i in range(n)]
    return X_sol, Z_sol, r_sol, model.ObjVal




In [3]:
# --- Example usage ---
n = 3
pauli_XZ = [ ([1,0,1], [0,1,1]),  ([1,1,0], [0,1,1]) ]
coeffs   = [ 0.5, -1.2 ]
X_tab, Z_tab, phases, optval = optimize_stabilizer(n, pauli_XZ, coeffs)

Restricted license - for non-production use only - expires 2026-11-23
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Debian GNU/Linux 12 (bookworm)")

CPU model: AMD Ryzen 9 PRO 5945 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 6 rows, 58 columns and 14 nonzeros
Model fingerprint: 0x0db3698f
Model has 29 quadratic constraints
Variable types: 0 continuous, 58 integer (35 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 3e+00]
  Objective range  [5e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e+00, 4e+00]
Found heuristic solution: objective 0.0000000
Presolve removed 3 rows and 9 columns
Presolve time: 0.00s
Presolved: 212 rows, 109 columns, 577 nonzeros
Variable types: 0 continuous, 109 integer (94 b

In [4]:
import numpy as np

I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# Build the Hamiltonian
H = (
    0.5 * np.kron(X, np.kron(Z, Y)) +
    -1.2 * np.kron(X, np.kron(Y, Z))
)

# Compute eigenvalues
eigvals = np.linalg.eigvalsh(H)
ground_energy = np.min(eigvals)
print("Ground state energy:", ground_energy)


Ground state energy: -1.7000000000000002


In [5]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize, claptonize_opt, claptonize_opt_scipy, claptonize_opt_ng, claptonize_opt_structured
from clapton.ansatzes import circular_ansatz, full_ansatz
from clapton.utils import Results
import numpy as np
import pickle
import time
import copy


from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.problems import ElectronicStructureProblem

from optuna.samplers import TPESampler, RandomSampler, GridSampler, CmaEsSampler, QMCSampler, GPSampler, NSGAIIISampler, NSGAIISampler


import nevergrad as ng
# from nevergrad.families import NonObjectOptimizer
# from nevergrad.optimizers import NGOpt
# from nevergrad.optimization.optimizerlib import EDA, ConfPSO, ParametrizedOnePlusOne, cGA, NoisySplit, MultiDiscrete, AXP, BFGSCMAPlus, CMA, CSEC11, Carola3, ChoiceBase, ForceMultiCobyla, \
#     LogMultiBFGSPlus, MEDA, MetaCMA, MultiBFGSPlus, MultiCobylaPlus, MultiSQPPlus, NGO, PCEDA, SPSA, SQPCMAPlus, Shiwa, SplitOptimizer, Wiz


In [6]:

def generate_h2o_geometry(bond_length: float) -> str:
    """
    Generate H2O geometry with both O–H bonds of equal length,
    forming an angle of ~104.5 degrees.
    """
    angle_deg = 104.5
    angle_rad = np.deg2rad(angle_deg / 2)

    x = bond_length * np.sin(angle_rad)
    z = bond_length * np.cos(angle_rad)

    geometry_str = (
        f"O 0.0 0.0 0.0; "
        f"H {x:.6f} 0.0 {z:.6f}; "
        f"H {-x:.6f} 0.0 {z:.6f}"
    )
    return geometry_str

def generate_linear_h6_geometry(bond_length: float) -> str:
    coords = []
    for i in range(6):
        z = i * bond_length
        coords.append(f"H 0.0 0.0 {z:.6f}")
    return "; ".join(coords)


def generate_ring_h6_geometry(bond_length: float) -> str:
    # Radius r from bond length of regular hexagon
    r = bond_length / (2 * np.sin(np.pi / 6))
    coords = []
    for i in range(6):
        angle = i * np.pi / 3  # 60° steps
        x = r * np.cos(angle)
        y = r * np.sin(angle)
        coords.append(f"H {x:.6f} {y:.6f} 0.0")
    return "; ".join(coords)


In [7]:
def binary_to_pauli_string(x_bits, z_bits):
    """Convert binary Pauli representation to string."""
    pauli_chars = []
    for x, z in zip(x_bits, z_bits):
        if x == 0 and z == 0:
            pauli_chars.append('I')
        elif x == 1 and z == 0:
            pauli_chars.append('X')
        elif x == 0 and z == 1:
            pauli_chars.append('Z')
        elif x == 1 and z == 1:
            pauli_chars.append('Y')
        else:
            raise ValueError(f"Invalid Pauli bit: x={x}, z={z}")
    return ''.join(pauli_chars)

def pauli_string_to_binary(pauli_str):
    """Convert string Pauli representation to binary ([x_bits], [z_bits])."""
    x_bits = []
    z_bits = []
    for p in pauli_str:
        if p == 'I':
            x_bits.append(0); z_bits.append(0)
        elif p == 'X':
            x_bits.append(1); z_bits.append(0)
        elif p == 'Z':
            x_bits.append(0); z_bits.append(1)
        elif p == 'Y':
            x_bits.append(1); z_bits.append(1)
        else:
            raise ValueError(f"Invalid Pauli character: {p}")
    return x_bits, z_bits



In [12]:
hamiltonian_strings = []
hamiltonian_strings += [f"H 0 0 0; H 0 0 {bond_length :.3f}" for bond_length in np.arange(0.37, 3.0, 0.1)]
# hamiltonian_strings += [f"Li 0 0 0; H 0 0 {bond_length :.3f}" for bond_length in np.arange(0.8, 4.8, 0.1)]
# hamiltonian_strings += [generate_h2o_geometry(bond_length) for bond_length in np.arange(0.5, 4.0, 0.1)]
# hamiltonian_strings += [generate_ring_h6_geometry(bond_length) for bond_length in np.arange(0.45, 3.6, 0.1)]
print(hamiltonian_strings)


for i, hamiltonian_string in enumerate(hamiltonian_strings):
    print(f"=================================")
    print(f"Starting {i+1}/{len(hamiltonian_strings)}: {hamiltonian_string}")
    print(f"=================================")
    driver = PySCFDriver(atom=hamiltonian_string, basis="sto-3g")
    problem = driver.run()
    second_q_op = problem.hamiltonian.second_q_op()

    mapper = ParityMapper()
    qubit_op = mapper.map(second_q_op)

    paulis_ = qubit_op.paulis.to_labels()
    coeffs_ = qubit_op.coeffs
    n = len(paulis_[0])
    
    if np.all(np.abs(coeffs_.imag) < 1e-10) is False:
        print(f"Skipping {hamiltonian_string} due to complex coefficients.")
        continue
    
    print(paulis_)
    paulis_ = [pauli_string_to_binary(paulis__) for paulis__ in paulis_]
    print(paulis_)
    coeffs_ = coeffs_.real.tolist()
    
    X_tab, Z_tab, phases, optval = optimize_stabilizer(n, paulis_, coeffs_)

['H 0 0 0; H 0 0 0.370', 'H 0 0 0; H 0 0 0.470', 'H 0 0 0; H 0 0 0.570', 'H 0 0 0; H 0 0 0.670', 'H 0 0 0; H 0 0 0.770', 'H 0 0 0; H 0 0 0.870', 'H 0 0 0; H 0 0 0.970', 'H 0 0 0; H 0 0 1.070', 'H 0 0 0; H 0 0 1.170', 'H 0 0 0; H 0 0 1.270', 'H 0 0 0; H 0 0 1.370', 'H 0 0 0; H 0 0 1.470', 'H 0 0 0; H 0 0 1.570', 'H 0 0 0; H 0 0 1.670', 'H 0 0 0; H 0 0 1.770', 'H 0 0 0; H 0 0 1.870', 'H 0 0 0; H 0 0 1.970', 'H 0 0 0; H 0 0 2.070', 'H 0 0 0; H 0 0 2.170', 'H 0 0 0; H 0 0 2.270', 'H 0 0 0; H 0 0 2.370', 'H 0 0 0; H 0 0 2.470', 'H 0 0 0; H 0 0 2.570', 'H 0 0 0; H 0 0 2.670', 'H 0 0 0; H 0 0 2.770', 'H 0 0 0; H 0 0 2.870', 'H 0 0 0; H 0 0 2.970']
Starting 1/27: H 0 0 0; H 0 0 0.370
['IIII', 'IIIZ', 'IIZZ', 'IIZI', 'IZZI', 'IZZZ', 'ZZII', 'ZZIZ', 'ZXIX', 'IXZX', 'ZXZX', 'IXIX', 'IZIZ', 'ZZZZ', 'ZIZI']
[([0, 0, 0, 0], [0, 0, 0, 0]), ([0, 0, 0, 0], [0, 0, 0, 1]), ([0, 0, 0, 0], [0, 0, 1, 1]), ([0, 0, 0, 0], [0, 0, 1, 0]), ([0, 0, 0, 0], [0, 1, 1, 0]), ([0, 0, 0, 0], [0, 1, 1, 1]), ([0, 0, 0, 0]

GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information